In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv('因子報酬敘述統計/factor_return_adj.csv').drop('Unnamed: 0',axis=1)
data = data.set_index('Date')


#半衰期(half-life)權重
def half(t,a):  #t=half-life, a=sample size
     lambd=0.5**(1/t)
     weightlist=[]
     for i in range(a-1,-1,-1): #a=539,b=-1,i=539、538、...0
         halflife=lambd**i
         weightlist.append(halflife)
     weightlist = np.array(weightlist)/sum(weightlist)   
     return weightlist

delta_weilist=[]
for i in range(1,16):
    delta_wei=(1-(i/(1+15)))
    delta_weilist.append(delta_wei)
    
#sam_size=1500
sam_size=540
#half=125
half_ =45 

#求解月風險所需矩陣
mat = data.shape[1]

deltalist_pp=[]
deltalist_mm=[]
for day in range(data.shape[0]-sam_size):
    data1=data.iloc[0+day:sam_size+day]
    deltalist_m=[]
    deltalist_p=[]
    for i in range(1,16): 
        Wv = np.zeros((sam_size,sam_size))
        Wv[np.arange(sam_size), np.arange(sam_size)] = half(half_,sam_size)
        Wc = np.zeros((sam_size,sam_size))
        Wc[np.arange(sam_size), np.arange(sam_size)] = half(half_,sam_size)
        Wv = Wv[i:,i:]
        Wc = Wc[i:,i:]
        M1 = data1.shift(i)[i:len(data1)] #0-538
        M2 = data1[i:len(data1)]          #1-539
        U1 = np.zeros((mat,mat))
        U1[np.arange(mat), np.arange(mat)] = 1
        U2 = np.ones((mat,mat))
        U2[np.arange(mat), np.arange(mat)] = 0
        Cd_p = Wv.dot(M1).T.dot(M2)*U1 + Wc.dot(M1).T.dot(M2)*U2
        Cd_m = Cd_p.T
        deltalist_p.append(Cd_p) 
        deltalist_m.append(Cd_m)
    deltalist_pp.append([data.index[sam_size+day],deltalist_p])
    deltalist_mm.append([data.index[sam_size+day],deltalist_m])
    print(day)
    
#求解日風險
port_varlist=[]
for day in range(data.shape[0]-sam_size):
    data1=data.iloc[0+day:sam_size+day]
    data2=data1-data1.describe().loc['mean']
     
    #EUE3S_var
    varT=(data2.T*half(half_,sam_size)).dot(data2)
    varT=varT/sum(half(half_,sam_size))
    var=np.diag(varT)
    
    #EUE3S_cor
    corT=(data2.T*half(half_,sam_size)).dot(data2)
    corT=corT/sum(half(half_,sam_size))
    #Cd
    Cd=corT.values.copy()
    np.fill_diagonal(Cd, var)
    #CdT=pd.DataFrame(Cd)
    port_varlist.append([data.index[sam_size+day],Cd])
    print(day) 

#求解月風險
Cmlist=[]    
for day in range(data.shape[0]-sam_size):  
    delta_times=[]
    for i in range(15): 
        d_times=delta_weilist[i]*( deltalist_pp[day][1][i]+ deltalist_mm[day][1][i] )
        delta_times.append(d_times)
    delta_sum=sum(delta_times)   
    Cm=(port_varlist[day][1] + delta_sum)*22
    Cmlist.append([data.index[sam_size+day], Cm])
    
#存起來
for i in range(data.shape[0]-sam_size):
    a=Cmlist[i][1]
    np.save('Common Factor Risk_Long\Cm'+str(i)+'.npy',a)